# First search algorithm

In [156]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
%matplotlib widget

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [157]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import jupyter_black
from IPython.display import display

jupyter_black.load()

# Project
from src.data_connectors import read_input_files
from src.data_preparation import search_space

In [158]:
instance = 336
instance = 217
instances_path = "../data/input/HRTInstances"
ins_x = read_input_files.read_file(
    os.path.join(instances_path, f"Instance_{instance}.txt")
)

In [159]:
ins_x.df_resources

,Type,Id
0,H,0
1,R,1
2,R,2


In [160]:
ins_x.df_setup

,Humans,Robots,WorkingSpaces
0,1,2,2


In [161]:
df_tasks = search_space.get_all_times_in_different_columns_per_task(ins_x)
df_tasks.head()

/Users/patriciarodrigues/Documents/GitHubPhD/HM_HRT/src/data_preparation/search_space.py:80: FutureWarning: pivot_table dropped a column because it failed to aggregate. This behavior is deprecated and will raise in a future version of pandas. Select only the columns that can be aggregated.
  df_times_per_resource = pd.pivot_table(


,MainJob,Job,0,1,2,Predecessors
0,0,1,13.0,1.0,NaN,[]
1,0,2,35.0,NaN,NaN,[1]
2,0,3,5.0,NaN,NaN,[2]
3,0,4,30.0,NaN,NaN,"[1, 2]"
4,0,5,99.0,NaN,NaN,"[1, 3]"


## Logic

In [162]:
from itertools import combinations


def find_possible_collaborations(list_resources, max_to: int = 3):
    list_combinations = []

    for n in range(1, max_to):
        list_combinations.extend(list(combinations(list_resources, n)))

    return list_combinations

In [163]:
def get_modes_columns(all_modes: list[tuple[int]]) -> list[str]:
    column_modes = []
    for mode in all_modes:
        if len(mode) == 1:
            column_modes.append(str(mode[0]))
        elif len(mode) == 2:
            column_modes.append(f"T:{mode[0]}-{mode[1]}")
    return column_modes

In [164]:
# Job | Resource | start_ts | end_ts
df_allocation = pd.DataFrame(columns=["Job", "Resource", "start_ts", "end_ts"])
# initialise dataframe
all_modes = find_possible_collaborations(ins_x.df_resources.Id.tolist())
mode_columns = get_modes_columns(all_modes)

for mode in mode_columns:
    df_allocation.loc[len(df_allocation.index)] = [0, mode, 0, 0]

df_allocation

,Job,Resource,start_ts,end_ts
0,0,0,0,0
1,0,1,0,0
2,0,2,0,0
3,0,T:0-1,0,0
4,0,T:0-2,0,0
5,0,T:1-2,0,0


In [173]:
def flatten(lst: list[list[str]]) -> list[str]:
    flattened_list = []
    for i in lst:
        if type(i) == list:
            flattened_list.extend(flatten(i))
        else:
            flattened_list.append(i)
    return flattened_list

In [174]:
flatten([[1], [1, 3, 4], [2, 3]])

[1, 1, 3, 4, 2, 3]

In [180]:
a = 1
while len(df_allocation) <= len(df_tasks):
    allocated_tasks = df_allocation.Job.tolist()
    # see what next tasks can be done:
    df_possible_tasks = df_tasks[
        df_tasks.Predecessors.apply(lambda x: set(x).issubset(allocated_tasks))
    ]

    new_tasks = [
        x for x in df_possible_tasks.Job.tolist() if x not in allocated_tasks
    ]
    print("next tasks can be done: ", new_tasks)

    # allocate the tasks to resources and append that information
    for new_task in new_tasks:
        print(f"which resource can do this task {new_task}? ")
        df_task = (
            df_tasks[df_tasks["Job"] == new_task]
            .drop(["MainJob", "Job", "Predecessors"], axis=1)
            .T
        )
        df_task.columns = ["Time"]
        display(df_task)

        # what is the min starting tine of the task?
        predecessor_tasks = flatten(
            df_tasks[df_tasks.Job == new_task].Predecessors.tolist()
        )
        print(predecessor_tasks)

        if len(predecessor_tasks) != 0:
            # get the sonner starting point
            starting_point_task = df_allocation[
                df_allocation.Job.isin(predecessor_tasks)
            ].end_ts.max()
            print(
                f"Task {new_task} can only be started after {starting_point_task} time"
            )
        else:
            starting_point_task = 0
            print(
                f"Task {new_task} can only be started after {starting_point_task} time"
            )

        # Allocate the tasks to the resource that will make the task to end sonner:
        possible_resources = [str(x) for x in df_task.dropna().index.tolist()]
        print(possible_resources)
        df_allocation_possible_resources = df_allocation[
            df_allocation.Resource.isin(possible_resources)
        ]
        display(df_allocation_possible_resources)

        resorce_that_ends_sonner = []
        # select the last entry per resource
        df_allocation_possible_resources_recent = (
            df_allocation_possible_resources.groupby("Resource")
            .max()
            .reset_index()
        )

        # What's the resource that will end sonner?
        df_allocation_possible_resources_recent_test = (
            df_allocation_possible_resources_recent.copy(deep=True)
        )

        for resource in df_task.index:
            # sum duration to the end of each possible resource, to know which can finish the task first
            previous_start_ts_of_resource = (
                df_allocation_possible_resources_recent[
                    df_allocation_possible_resources_recent.Resource
                    == str(resource)
                ].start_ts.max()
            )
            previous_end_ts_of_resource = (
                df_allocation_possible_resources_recent[
                    df_allocation_possible_resources_recent.Resource
                    == str(resource)
                ].end_ts.max()
            )

            df_allocation_possible_resources_recent_test.loc[
                len(df_allocation_possible_resources_recent_test.index)
            ] = [
                new_task,
                resource,
                max(previous_start_ts_of_resource, starting_point_task),
                max(previous_end_ts_of_resource, starting_point_task)
                + df_task.iloc[resouce_min].Time,
            ]
        display(df_allocation_possible_resources_recent_test)
        print("HERE")

        # resource that takes less time
        resouce_min = int(df_task.Time.astype(float).min())
        print(resouce_min)

        print(previous_start_ts_of_resource, previous_end_ts_of_resource)

        df_allocation_possible_resources_recent.loc[
            len(df_allocation_possible_resources_recent.index)
        ] = [
            new_task,
            resouce_min,
            previous_start_ts_of_resource,
            previous_end_ts_of_resource + df_task.iloc[resouce_min].Time,
        ]
        display(df_allocation_possible_resources_recent)

        print("=============")
    a += 1
    if a > 1:
        break

next tasks can be done:  [1, 21]
which resource can do this task 1? 


,Time
0,13.0
1,1.0
2,NaN


[]
Task 1 can only be started after 0 time
['0', '1']


,Job,Resource,start_ts,end_ts
0,0,0,0,0
1,0,1,0,0


,Resource,Job,start_ts,end_ts
0,0,0.0,0.0,0.0
1,1,0.0,0.0,0.0
2,1.0,0.0,0.0,1.0
3,1.0,1.0,0.0,1.0
4,1.0,2.0,NaN,NaN


...

[]
Task 21 can only be started after 0 time
['0', '2']


,Job,Resource,start_ts,end_ts
0,0,0,0,0
2,0,2,0,0


,Resource,Job,start_ts,end_ts
0,0,0.0,0.0,0.0
1,2,0.0,0.0,0.0
2,21.0,0.0,0.0,NaN
3,21.0,1.0,NaN,NaN
4,21.0,2.0,0.0,NaN


HERE
1
0 0


,Resource,Job,start_ts,end_ts
0,0,0.0,0.0,0.0
1,2,0.0,0.0,0.0
2,21.0,1.0,0.0,NaN
